# Dataset Preprocessing
In this notebook, the goal is to process the dataset so that the dataset is ready to be used for training. To achieve this goal, here are the steps that will be performed in this notebook:
1. Combine the CSV files of the dataset into one dataset
2. Preliminary analysis of the dataset
3. Dataset cleaning - clean missing values, duplicates, etc
4. Further processing, includes addressing the high class imbalance problem and rename the columns name (if necessary)
5. Save the processed dataset

note: The notebook will be seperated into two subsections, where the training dataset and the testing dataset will be processed separately. However, same steps will be applied to both dataset.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import the packages to be used
import os       # to create directories and remove files
import numpy as np
import pandas as pd
import random

# set the random seed to ensure the result is reproducible
random.seed(10)
np.random.seed(10)

# LUFlow Dataset (2022)

For the implementation using the LUFlow dataset, the data collected in June 2022 is being used as training dataset.

## Step 1. Combine the files of the dataset

In [3]:
luflow_csv_files = ('2022.06.12',
                    '2022.06.13',
                    '2022.06.14',
                    )

In [14]:
# Define the function to combine seperate files of a dataset
# Besides combining the files, it has two additional functionality:
# 1. Replace the replacement character (\uFFFD) with '-', which is needed for the CIC-IDS2017 dataset
# 2. Down sample the dataset. If the reduce_sample_size parameter is set to true,
#    only 10% of the dataset will be randomly selected and saved.
def combine_csv_files(dataset: str, file_names: tuple, reduce_sample_size: bool = False):

    # create a new directory to place the combined dataset
    os.makedirs('./Dataset/dataset_combined', exist_ok=True)

    # remove the dataset if it already exist
    merged_dataset_directory = f'/content/drive/MyDrive/VIT/Dataset/LuFlow/dataset_combined/{dataset}.csv'
    if os.path.isfile(merged_dataset_directory):
        os.remove(merged_dataset_directory)
        print(f'original file({merged_dataset_directory}) has been removed')

    for (file_index, file_name) in enumerate(file_names):
        with open(f"/content/drive/MyDrive/VIT/Dataset/{dataset}/{file_name}.csv", 'r') as file, open(merged_dataset_directory, 'a') as out_file:
            for (line_index, line) in enumerate(file):

                # only the header of the first file will be taken
                if 'Label' in line or 'label' in line:
                    if file_index != 0 or line_index != 0:
                        continue
                elif reduce_sample_size:
                    if random.randint(1, 10) > 1:
                        continue
                # replace the replacement character (\uFFFD) with '-' if exist
                out_file.write(line.replace(' ï¿½ ', '-'))


In [15]:
combine_csv_files(dataset='LuFlow',
                    file_names=luflow_csv_files,
                    reduce_sample_size=True)

## Step 2. Preliminary analysis

In [17]:
# read the dataset
luflow2022 = pd.read_csv('/content/drive/MyDrive/VIT/Dataset/LuFlow/dataset_combined/LuFlow.csv')
luflow2022.head()

,avg_ipt,bytes_in,bytes_out,dest_ip,dest_port,entropy,num_pkts_out,num_pkts_in,proto,src_ip,src_port,time_end,time_start,total_entropy,label,duration
0,0.000000e+00,0,0,786,31306.0,0.00000,1,0,6,786,47613.0,1655074782132491,1655074782132491,0.000000e+00,outlier,0.000000
1,0.000000e+00,0,0,786,15402.0,0.00000,1,0,6,786,47613.0,1655074853394147,1655074853394147,0.000000e+00,outlier,0.000000
2,0.000000e+00,0,0,786,26469.0,0.00000,1,0,6,786,47613.0,1655074763683012,1655074763683012,0.000000e+00,outlier,0.000000
3,0.000000e+00,8,8,786,NaN,3.00000,1,1,1,786,NaN,165507476660575,165507476660562,4.800000e+01,malicious,0.000130
4,5.052937e+07,38636,8014,786,9200.0,32.04859,225,159,6,786,50422.0,1655075048115423,1655075018601277,1.495067e+06,benign,29.514147


In [18]:
print(f"Number of rows: {luflow2022.shape[0]}")
print(f"Number of columns: {luflow2022.shape[1]}")

Number of rows: 106967
Number of columns: 16


In [19]:
print("Columns in the dataset:")
luflow2022.columns

Columns in the dataset:


Index(['avg_ipt', 'bytes_in', 'bytes_out', 'dest_ip', 'dest_port', 'entropy',
       'num_pkts_out', 'num_pkts_in', 'proto', 'src_ip', 'src_port',
       'time_end', 'time_start', 'total_entropy', 'label', 'duration'],
      dtype='object')

In [20]:
print('Class distribution:')
luflow2022['label'].value_counts()

Class distribution:


benign       51773
outlier      44239
malicious    10955
Name: label, dtype: int64

From the class distribution, we can see that the LUFlow 2022 dataset is imbalance. The benign samples account for 48% of total samples while malicious only account for 11%.

In [21]:
print('Class distribution (normalized):')
luflow2022['label'].value_counts()/luflow2022.shape[0]*100

Class distribution (normalized):


benign       48.400909
outlier      41.357615
malicious    10.241476
Name: label, dtype: float64

### Check for null value

Count the number of rows include null values in each column.

In [ ]:
luflow2022_null_count = luflow2022.isnull().sum()
luflow2022_null_count = luflow2022_null_count[luflow2022_null_count > 0]
print(f"Rows contain null value: \n{luflow2022_null_count}\n")

luflow2020_null_count = luflow2022_null_count / luflow2022.shape[0] * 100
print(f"Rows contain null value (percentage): \n{luflow2022_null_count}\n")

Rows contain null value: 
dest_port    29268
src_port     29268
dtype: int64

Rows contain null value (percentage): 
dest_port    1.167754
src_port     1.167754
dtype: float64



### Check for infinity value

Check for number of rows that include null value

In [22]:
print('Number of samples contains infinity value:')
np.isinf(luflow2022.iloc[:, :-2]).any(axis=1).sum()

Number of samples contains infinity value:


0

### Check for columns that contain string values

Check for columns that is type `object`, which indicate the columns contain string value. The aim is to find if any column contain numeric and alphabetic value. Such column often include string value like '?' to indicate missing value, thus needed to be cleaned.

In [23]:
luflow2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106967 entries, 0 to 106966
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        106967 non-null  float64
 1   bytes_in       106967 non-null  int64  
 2   bytes_out      106967 non-null  int64  
 3   dest_ip        106967 non-null  int64  
 4   dest_port      96533 non-null   float64
 5   entropy        106967 non-null  float64
 6   num_pkts_out   106967 non-null  int64  
 7   num_pkts_in    106967 non-null  int64  
 8   proto          106967 non-null  int64  
 9   src_ip         106967 non-null  int64  
 10  src_port       96533 non-null   float64
 11  time_end       106967 non-null  int64  
 12  time_start     106967 non-null  int64  
 13  total_entropy  106967 non-null  float64
 14  label          106967 non-null  object 
 15  duration       106967 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 13.1+ MB


From here, we can see that only the 'label' column is of type `object`. As the 'label' column store the label of each sample using `string`, it has no problem with the column.

### Check for duplicates

In [24]:
# check for duplicated column
luflow2022.columns[luflow2022.columns.value_counts() > 1]

Index([], dtype='object')

In [26]:
luflow_duplicates = luflow2022[luflow2022.duplicated()]
print(f"{luflow_duplicates.shape[0]} rows are duplicates")
print(f"{luflow_duplicates.shape[0]/luflow2022.shape[0]*100:.2f}% of rows are duplicates")

298 rows are duplicates
0.28% of rows are duplicates


## Step 3. Dataset cleaning

From the analysis in Step 2, it has been discovered that the dataset contains a small amount of missing value and duplicates. Since those problematic rows only account for a small portion of the entire dataset, those rows are simply removed.

In [27]:
# remove rows contain missing value
luflow2022 = luflow2022.dropna(how='any')
luflow2022.shape

(96533, 16)

In [28]:
luflow2022 = luflow2022.drop_duplicates()
luflow2022.shape

(96325, 16)

## Step 4. Dataset preparation

In this step, the main goal is to prepare the dataset to be ready for training. For the LUFlow dataset, there are two tasks to be completed:
* Remove outlier - The LUFlow dataset contains an "outlier" class. In the outlier class, it contains samples that are malicious, but could also be benign. Hence, the outliers are not meaningful as they sits in the grey area between malicious and benign class.
* Balance the class distribution - The benign samples are slightly more than the malicious samples, which could result in the models bias towards the benign samples. Since we have a large number of samples, we will downsampling the benign samples, so that the ratio between benign and malicious samples is 1:1.

In [29]:
attack = luflow2022[luflow2022['label']=='malicious']
benign = luflow2022[luflow2022['label']=='benign'].sample(n=len(attack)).reset_index(drop=True)

luflow2022_exclude_outlier = pd.concat([attack, benign])
del attack
del benign

luflow2022_exclude_outlier['label'].value_counts()

malicious    7607
benign       7607
Name: label, dtype: int64

## Step 5. Save the dataset

In [30]:
# function to save the cleaned dataset
def save_cleaned_dataset(dataframe: pd.DataFrame,dataset: str, tag: str = ""):
    # create a new directory to save the cleaned dataset
    os.makedirs('/content/drive/MyDrive/VIT/Dataset/LuFlow/dataset_cleaned', exist_ok=True)

    if not(tag == ""):
        tag = "_" + tag

    dataframe.to_csv(f'/content/drive/MyDrive/VIT/Dataset/LuFlow/dataset_cleaned/{dataset}{tag}.csv', index=False)

In [31]:
save_cleaned_dataset(dataframe=luflow2022_exclude_outlier, dataset='LUFlow')

# LUFlow Dataset (2021)

For the implementation using the LUFlow dataset, the data collected in February 2021 is being used as testing dataset.

In [32]:
luflow2021_csv_files = ('2021.02.17',
                        )

In [34]:
combine_csv_files(dataset='LuFlow2021',
                    file_names=luflow2021_csv_files,
                    reduce_sample_size=True)

In [37]:
luflow2021 = pd.read_csv('/content/drive/MyDrive/VIT/Dataset/LuFlow/dataset_combined/LuFlow2021.csv')
luflow2021.head()

,avg_ipt,bytes_in,bytes_out,dest_ip,dest_port,entropy,num_pkts_out,num_pkts_in,proto,src_ip,src_port,time_end,time_start,total_entropy,label,duration
0,466.857143,34,29,786,5900.0,5.139898,10,13,6,786,38663.0,1613521515164186,1613521511597962,323.81360,outlier,3.566224
1,46.857143,34,29,786,5900.0,5.096170,7,10,6,786,19182.0,1613521509058904,161352150846949,321.05872,outlier,0.589414
2,11.687500,2231,3196,786,22.0,7.611605,22,24,6,786,31182.0,1613521510969683,1613521510579643,41308.18000,malicious,0.390040
3,149.750000,270,191,786,445.0,4.531242,6,6,6,786,58992.0,1613521522530237,1613521521360967,2088.90260,malicious,1.169270
4,49.428571,34,29,786,5900.0,4.988950,8,10,6,786,53110.0,1613521524087434,1613521523508882,314.30386,outlier,0.578552


In [38]:
print(f"Number of rows: {luflow2021.shape[0]}")
print(f"Number of columns: {luflow2021.shape[1]}")

Number of rows: 59440
Number of columns: 16


In [39]:
print("Columns in the dataset:")
luflow2021.columns

Columns in the dataset:


Index(['avg_ipt', 'bytes_in', 'bytes_out', 'dest_ip', 'dest_port', 'entropy',
       'num_pkts_out', 'num_pkts_in', 'proto', 'src_ip', 'src_port',
       'time_end', 'time_start', 'total_entropy', 'label', 'duration'],
      dtype='object')

In [40]:
print('Class distribution')
luflow2021['label'].value_counts()

Class distribution


benign       34920
malicious    15423
outlier       9097
Name: label, dtype: int64

From the class distribution, we can see that the LUFlow 2021 dataset is also imbalance. The benign samples account for 58% of total samples while malicious only account for 25%.

In [42]:
print('Class distribution (normalized):')
luflow2021['label'].value_counts()/luflow2021.shape[0]*100

Class distribution (normalized):


benign       58.748318
malicious    25.947174
outlier      15.304509
Name: label, dtype: float64

### Check for null value

In [43]:
luflow2021_null_count = luflow2021.isnull().sum()
luflow2021_null_count = luflow2021_null_count[luflow2021_null_count > 0]
print(f"Rows contain null value: \n{luflow2021_null_count}\n")

luflow2021_null_count = luflow2021_null_count / luflow2021.shape[0] * 100
print(f"Rows contain null value (percentage): \n{luflow2021_null_count}\n")

Rows contain null value: 
dest_port    145
src_port     145
dtype: int64

Rows contain null value (percentage): 
dest_port    0.243943
src_port     0.243943
dtype: float64



### Check for infinity value

In [44]:
print('Number of samples contains infinity value:')
np.isinf(luflow2021.iloc[:, :-2]).any(axis=1).sum()

Number of samples contains infinity value:


0

### Check for columns that contain string values

In [45]:
luflow2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59440 entries, 0 to 59439
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   avg_ipt        59440 non-null  float64
 1   bytes_in       59440 non-null  int64  
 2   bytes_out      59440 non-null  int64  
 3   dest_ip        59440 non-null  int64  
 4   dest_port      59295 non-null  float64
 5   entropy        59440 non-null  float64
 6   num_pkts_out   59440 non-null  int64  
 7   num_pkts_in    59440 non-null  int64  
 8   proto          59440 non-null  int64  
 9   src_ip         59440 non-null  int64  
 10  src_port       59295 non-null  float64
 11  time_end       59440 non-null  int64  
 12  time_start     59440 non-null  int64  
 13  total_entropy  59440 non-null  float64
 14  label          59440 non-null  object 
 15  duration       59440 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 7.3+ MB


Just like the LUFlow 2020 dataset, only the 'label' column is of type `object`, so there will be no cleaning needed

## Check for duplicates

In [46]:
# check for duplicated column
luflow2021.columns[luflow2021.columns.value_counts() > 1]

Index([], dtype='object')

In [47]:
luflow2021_duplicates = luflow2021[luflow2021.duplicated()]
print(f"{luflow2021_duplicates.shape[0]} rows are duplicates")
print(f"{luflow2021_duplicates.shape[0]/luflow2021.shape[0]*100:.2f}% of rows are duplicates")


1 rows are duplicates
0.00% of rows are duplicates


## Step 3. Dataset cleaning

Just like the LUFlow 2020 dataset, the LUFlow 2021 dataset contains a small amount of missing value and duplicated rows. Those problematic rows in this dataset are also removed.

In [48]:
luflow2021 = luflow2021.dropna(how='any')
luflow2021.shape

(59295, 16)

In [49]:
luflow2021 = luflow2021.drop_duplicates()
luflow2021.shape

(59294, 16)

## Step 4. Dataset preparation

Same as before, the outliers are removed and the class distribution are balanced in this step.

In [50]:
attack = luflow2021[luflow2021['label'] == 'malicious']
benign = luflow2021[luflow2021['label'] == 'benign'].sample(n=len(attack)).reset_index(drop=True)

luflow2021_exclude_outlier = pd.concat([attack, benign])
del attack
del benign

luflow2021_exclude_outlier['label'].value_counts()

malicious    15314
benign       15314
Name: label, dtype: int64

## Step 5. Save the dataset

In [51]:
save_cleaned_dataset(dataframe=luflow2021_exclude_outlier, dataset='LUFlow2021')